In [1]:
import numpy as np
import pandas as pd
import time

index_to_base = {0: 'A', 1: 'C', 2: 'U', 3: 'G'}
base_to_number={'A':0, 'C':1, 'U':2, 'G':3, 'T':2}
db_to_bin = {'.': '00', '(': '10', ')': '01', '_': '00', '[': '10', ']': '01'}


def sequence_to_structure(seq, GPmap):
   return get_dotbracket_from_int(GPmap[sequence_str_to_int(seq)])

def sequence_str_to_int(sequence):
   """convert between sequence representations:
   from biological four-letter string to a tuple of integers from 0-3;
   motive: these integers can be used as array tuples"""
   return tuple([base_to_number[b] for b in sequence])

def sequence_tuple_to_str(sequence):
    return "".join([index_to_base[i] for i in sequence])

def get_dotbracket_from_int(structure_int):
   """ retrieve the full dotbracket string from the integer representation"""
   dotbracketstring = ''
   bin_to_db = {'10': '(', '00': '.', '01': ')'}
   structure_bin = bin(structure_int)[3:] # cut away '0b' and the starting 1
   assert len(structure_bin) % 2 == 0
   for indexpair in range(0, len(structure_bin), 2):
      dotbracketstring = dotbracketstring + bin_to_db[structure_bin[indexpair]+structure_bin[indexpair+1]]
   return dotbracketstring

def get_sequence_from_int(sequence_int):
    sequence_strukture = ''
    sequence_bin = bin(sequence_int)[2:] # cut away '0b'
    # fill in missing leading zeros
    while len(sequence_bin) < 26:
        sequence_bin = "0" + sequence_bin
    for i in range(0, len(sequence_bin), 2):
        sequence_strukture = sequence_strukture + index_to_base[int(sequence_bin[i]+sequence_bin[i+1], 2)]
    return sequence_strukture

def point_mutations(genotype):
    genotype = list(genotype)
    return (tuple(genotype[:i]+[g]+genotype[i+1:]) for i in range(len(genotype)) for g in set((0,1,2,3)) - {genotype[i]})

def neighbours(genotype, neutral_set):
    return [_ for _ in point_mutations(genotype) if _ in neutral_set]

def dfs(neutral_set, start=[]):
    if not start:
        start = neutral_set[0]
    U = [start]
    V = [start]
    while U:
        current_node = U[0]
        for n in neighbours(current_node, neutral_set):
            if n not in V:
                U.append(n)
                V.append(n)
        U = U[1:]
    return V
    
##################################################################################################
##################################################################################################
print("RNA GP map: save data")
##################################################################################################
##################################################################################################

L = 13

#################################################
print("load")
#################################################
GPmap = np.load('GPmap_L'+str(L)+'mfe.npy')

#################################################
print("now we can use the GP map")
#################################################
seq_test = ''.join([np.random.choice(['A', 'G', 'C', 'G']) for i in range(L)])
print('this is how we can look up the structure for a sequence', seq_test, sequence_to_structure(seq_test, GPmap))

RNA GP map: save data
load
now we can use the GP map
this is how we can look up the structure for a sequence GAGCCCGGGGGCC ..(((....))).


## Finden der neutral Sets

In [2]:
# Sets an gleichen Phänotypen finden
unique_phaenotypes_int = np.unique(GPmap)

unique_phaenotypes_dotbracket = []
for i in unique_phaenotypes_int:
    unique_phaenotypes_dotbracket.append(get_dotbracket_from_int(i))
    
"""# Die zugehörigen Genotypen finden
neutral_set_dict = {}
for i in unique_phaenotypes_int:
    genotypys_weird_format = np.where(GPmap==i)
    genotypes = list(zip(*genotypys_weird_format))
    neutral_set_dict[i] = genotypes
    
print(neutral_set_dict[109052224][0:10])
"""
length_test = 0
# Die zugehörigen Genotypen finden
natural_set_df = pd.DataFrame()
for i in unique_phaenotypes_int:
    a = time.time()
    genotypys_weird_format = np.where(GPmap==i)
    a_b = time.time()
    temp_2 = zip(*genotypys_weird_format)
    b = time.time()
    temp_test = list(temp_2)[:10000]
    b_c = time.time()
    genotypes = pd.Series(temp_test)
    c = time.time()
    natural_set_df = pd.concat([natural_set_df, genotypes], axis=1)
    d = time.time()
    
    length_test += len(genotypys_weird_format[0])
    
    print(f"Laufzeit zum phänotype {i} = {get_dotbracket_from_int(i)} (Anzahl genotyps: {len(genotypys_weird_format[0])}) -- genotyps finden: {(a_b-a):.2f}s -- genotypes transopieren(zip): {(b-a_b):.2f}s -- als liste :{(b_c-b):.2f}s -- daten in pd Series: {(c - b_c):.2f}s -- in pd Dataframe: {(d-c):.2f}s")

natural_set_df.columns = unique_phaenotypes_int

print("----------------------------------------------------")
print(f"Länge extrahierte Daten: {length_test}, Länge ursprüngliche Daten: {4**13}")
print("----------------------------------------------------")

natural_set_df.to_pickle("neutral_sets_data.pkl")

natural_set_df.head()

Laufzeit zum phänotype 67108864 = ............. (Anzahl genotyps: 54439172) -- genotyps finden: 2.27s -- genotypes transopieren(zip): 0.00s -- als liste :61.11s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.00s
Laufzeit zum phänotype 67149829 = .....((....)) (Anzahl genotyps: 86933) -- genotyps finden: 0.28s -- genotypes transopieren(zip): 0.10s -- als liste :0.06s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.00s
Laufzeit zum phänotype 67272709 = ....((.....)) (Anzahl genotyps: 48792) -- genotyps finden: 0.20s -- genotypes transopieren(zip): 0.00s -- als liste :0.04s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.00s
Laufzeit zum phänotype 67272724 = ....((....)). (Anzahl genotyps: 297983) -- genotyps finden: 0.21s -- genotypes transopieren(zip): 0.00s -- als liste :0.20s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.00s
Laufzeit zum phänotype 67280917 = ....(((...))) (Anzahl genotyps: 122748) -- genotyps finden: 0.21s -- genotypes transopieren(zip): 0.00s -- als li

Laufzeit zum phänotype 111149077 = (((.......))) (Anzahl genotyps: 165008) -- genotyps finden: 0.21s -- genotypes transopieren(zip): 0.00s -- als liste :0.09s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.01s
Laufzeit zum phänotype 111149140 = (((......))). (Anzahl genotyps: 587796) -- genotyps finden: 0.22s -- genotypes transopieren(zip): 0.00s -- als liste :0.34s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.01s
Laufzeit zum phänotype 111149392 = (((.....))).. (Anzahl genotyps: 507745) -- genotyps finden: 0.21s -- genotypes transopieren(zip): 0.00s -- als liste :0.29s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.01s
Laufzeit zum phänotype 111150400 = (((....)))... (Anzahl genotyps: 774935) -- genotyps finden: 0.23s -- genotypes transopieren(zip): 0.00s -- als liste :0.45s -- daten in pd Series: 0.00s -- in pd Dataframe: 0.01s
Laufzeit zum phänotype 111154432 = (((...))).... (Anzahl genotyps: 370283) -- genotyps finden: 0.21s -- genotypes transopieren(zip): 0.00s -- al

,67108864,67149829,67272709,67272724,67280917,67764229,67764244,67764304,67797013,67797076,...,111149140,111149392,111150400,111154432,111673429,111673669,111673684,111674644,111674704,111804757
0,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","(0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 3, 3)","(0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 0)","(0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 3, 3, 3)","(0, 0, 0, 1, 0, 1, 0, 3, 2, 3, 2, 2, 3)","(0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 0)","(0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 0, 0)","(0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 2)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 0)",...,"(0, 0, 0, 1, 0, 3, 2, 3, 2, 2, 2, 2, 0)","(0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 2, 0, 0)","(0, 0, 1, 1, 2, 1, 3, 3, 2, 2, 0, 0, 0)","(0, 1, 1, 0, 0, 0, 3, 3, 2, 0, 0, 0, 0)","(0, 0, 0, 1, 3, 0, 0, 0, 0, 3, 2, 2, 2)","(0, 1, 1, 1, 0, 0, 0, 0, 3, 3, 1, 3, 2)","(0, 0, 0, 1, 0, 0, 0, 0, 3, 2, 2, 2, 0)","(1, 1, 1, 1, 0, 0, 0, 3, 3, 0, 3, 3, 0)","(0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 2, 0, 0)","(0, 0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 2, 2)"
1,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)","(0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 0, 0, 1, 2, 3, 3)","(0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 3)","(0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 3, 3, 3)","(0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 2, 3, 3)","(0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 1)","(0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 0, 1)","(0, 0, 0, 0, 0, 1, 2, 0, 1, 3, 3, 2, 2)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 1)",...,"(0, 0, 0, 1, 0, 3, 2, 3, 2, 2, 2, 2, 3)","(0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 2, 0, 1)","(0, 0, 1, 1, 2, 1, 3, 3, 2, 2, 0, 0, 1)","(0, 1, 1, 0, 0, 0, 3, 3, 2, 0, 0, 0, 1)","(0, 0, 0, 1, 3, 0, 0, 0, 3, 3, 2, 2, 2)","(0, 1, 1, 1, 0, 0, 0, 1, 3, 3, 1, 3, 2)","(0, 0, 0, 1, 0, 0, 0, 0, 3, 2, 2, 2, 3)","(1, 1, 1, 1, 0, 0, 0, 3, 3, 0, 3, 3, 1)","(0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 2, 0, 1)","(0, 0, 0, 1, 1, 0, 0, 2, 3, 3, 2, 2, 2)"
2,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2)","(0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 0, 0, 2, 2, 3, 3)","(0, 0, 0, 0, 0, 1, 2, 1, 1, 3, 3, 2, 3)","(0, 0, 0, 0, 1, 1, 1, 0, 0, 3, 3, 3, 3)","(0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 2, 3, 3)","(0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 0, 2)","(0, 0, 0, 0, 0, 1, 2, 1, 1, 3, 3, 2, 2)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 3)",...,"(0, 0, 1, 3, 0, 0, 0, 0, 0, 3, 2, 2, 0)","(0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 2, 0, 2)","(0, 0, 1, 1, 2, 1, 3, 3, 2, 2, 0, 0, 2)","(0, 1, 1, 0, 0, 0, 3, 3, 2, 0, 0, 0, 2)","(0, 0, 0, 1, 3, 0, 0, 1, 0, 3, 2, 2, 2)","(0, 1, 1, 1, 0, 0, 0, 2, 3, 3, 1, 3, 2)","(0, 0, 0, 1, 0, 0, 0, 1, 3, 2, 2, 2, 0)","(1, 1, 1, 1, 0, 0, 0, 3, 3, 0, 3, 3, 2)","(0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 2, 0, 2)","(0, 0, 0, 1, 1, 0, 0, 3, 3, 3, 2, 2, 2)"
3,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3)","(0, 0, 0, 0, 0, 1, 1, 1, 2, 0, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 0, 0, 3, 2, 3, 3)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 0)","(0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 3, 3, 3)","(0, 0, 0, 1, 1, 2, 0, 0, 0, 2, 2, 3, 3)","(0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 3, 3, 0)","(0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 0, 3)","(0, 0, 0, 0, 0, 1, 2, 3, 1, 3, 3, 2, 2)","(0, 0, 0, 0, 1, 1, 0, 0, 2, 3, 3, 2, 0)",...,"(0, 0, 1, 3, 0, 0, 0, 0, 0, 3, 2, 2, 3)","(0, 1, 1, 0, 0, 0, 0, 0, 3, 3, 2, 0, 3)","(0, 0, 1, 1, 2, 1, 3, 3, 2, 2, 0, 0, 3)","(0, 1, 1, 0, 0, 0, 3, 3, 2, 0, 0, 0, 3)","(0, 0, 0, 1, 3, 0, 0, 1, 3, 3, 2, 2, 2)","(0, 1, 1, 1, 0, 0, 1, 0, 3, 3, 1, 3, 2)","(0, 0, 0, 1, 0, 0, 0, 1, 3, 2, 2, 2, 3)","(1, 1, 1, 1, 0, 0, 0, 3, 3, 1, 3, 3, 0)","(0, 0, 1, 1, 0, 0, 0, 3, 3, 2, 2, 0, 3)","(0, 0, 0, 1, 1, 0, 1, 0, 3, 3, 2, 2, 2)"
4,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)","(0, 0, 0, 0, 0, 1, 1, 1, 2, 1, 3, 3, 3)","(0, 0, 0, 0, 1, 1, 2, 0, 1, 0, 2, 3, 3)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 1)","(0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 3, 3, 3)","(0, 0, 0, 1, 1, 2, 0, 0, 0, 3, 2, 3, 3)","(0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 3, 3, 1)","(0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 3, 0)","(0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 2)","(0, 0, 0, 0, 1, 1, 0, 0, 2, 3, 3, 2, 1)"

## Finden der neutral Component

In [3]:
a = time.time()
data_series = natural_set_df[67797013].dropna()
data_series = data_series.tolist()
list_comp = []
b = time.time()
corrector = 0
while data_series:
    d = time.time()
    print("starting Set to", data_series[0], " ---- ", len(data_series))
    neutral_components = dfs(data_series)
    list_comp.append(neutral_components)
    e = time.time()
    print("neutral component:", neutral_components[:10])
    for j in neutral_components:
        data_series.remove(j)
    f = time.time()
    print(f"algorithmus: {e-d}, listdrop: {f-e}")
    print("--------------")
    

c = time.time()
temp = 0
for i in list_comp:
    temp += len(i)

print(len(natural_set_df[67797013].dropna().tolist()), temp)

print(f"Zeiten: daten: {b-a:.2f}s, algorithmus: {c-b:.2f}s")

starting Set to (0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 2)  ----  10000
neutral component: [(0, 0, 0, 0, 0, 1, 1, 2, 1, 3, 3, 2, 2), (0, 0, 1, 0, 0, 1, 1, 2, 1, 3, 3, 2, 2), (0, 0, 0, 3, 0, 1, 1, 2, 1, 3, 3, 2, 2), (0, 0, 0, 0, 3, 1, 1, 2, 1, 3, 3, 2, 2), (0, 0, 0, 3, 3, 1, 1, 2, 1, 3, 3, 2, 2), (0, 0, 0, 3, 0, 1, 3, 2, 1, 3, 3, 2, 2), (0, 0, 0, 3, 0, 1, 1, 2, 1, 3, 3, 2, 1), (0, 0, 0, 0, 3, 1, 1, 2, 1, 3, 3, 1, 2), (0, 0, 0, 3, 3, 1, 1, 2, 1, 3, 3, 1, 2), (0, 0, 0, 3, 3, 1, 1, 2, 1, 3, 3, 2, 1)]
algorithmus: 41.4627959728241, listdrop: 0.8256049156188965
--------------
starting Set to (0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 2)  ----  8306
neutral component: [(0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 2), (0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 3, 3, 2), (0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 3, 3, 2), (0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 3, 3, 2), (0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 3, 3, 2), (0, 0, 0, 0, 1, 1, 3, 0, 0, 0, 3, 3, 2), (0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 3, 3, 2), (0, 0, 0, 0, 1, 1, 0, 2, 0, 0, 3, 3, 2), (0, 0, 0, 